In [2]:
print("ok")

ok


### Import Libraries

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
# import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [12]:
PINECONE_API_KEY = "bb77b29f-7fe6-4390-9510-cec577216f50"
PINECONE_API_ENV = "quickstart"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
#Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY,
#               environment=PINECONE_API_ENV)

# index_name="medical-chatbot"

# #Creating Embeddings for Each of The Text Chunks & storing
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# #If we already have an index we can load it like this
# docsearch=Pinecone.from_existing_index(index_name, embeddings)

# query = "What are Allergies"

# docs=docsearch.similarity_search(query, k=3)

# print("Result", docs)

In [17]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)  

 
index_name="medical-chatbot"
index = pc.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7067}},
 'total_vector_count': 7067}

In [54]:
# query_result = embeddings.embed_query("Hello world")
# print("Length", len(query_result))

query = "What are Allergies?"

# create the query vector
xq = embeddings.embed_query(query)

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True, namespace="real", include_values=True)
xc

{'matches': [{'id': '1437',
              'metadata': {'text': 'the itchy, scratchy nose, eyes, and throat '
                                   'common inallergic rhinitis .\n'
                                   'The particular allergens to which a person '
                                   'is sensi-'},
              'score': 0.695547402,
              'values': [-0.00302625983,
                         -0.0837671161,
                         -0.00527245179,
                         -0.0259764455,
                         0.0590001196,
                         0.00638939952,
                         0.0865671113,
                         0.0559244417,
                         -0.0915432349,
                         0.0574495047,
                         0.0634039938,
                         -0.0939055234,
                         -0.0315014459,
                         0.0907024816,
                         0.00618988,
                         0.011197431,
                         -

In [55]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.7: the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .
The particular allergens to which a person is sensi-
0.69: GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE
0.68: allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the following:
• nuts, especially peanuts, walnuts, and brazil nuts
• fish, mollusks, and shellfish• eggs• wheat• milk• food 

In [58]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [62]:
# query = "What are Allergies?"

# # create the query vector
# xq = embeddings.embed_query(query)

# # now query
# xc = index.query(vector=xq, top_k=5, include_metadata=True, namespace="real")

# vectorstore.similarity_search(
#     xc,  # our search query
# )

query = "when was the college of engineering in the University of Notre Dame established?"

xq = embeddings.embed_query(query)
xc = index.query(vector=xq, top_k=5, namespace="real")

vectorstore.similarity_search(
    query,  # our search query
    k=3,  # return 3 most relevant docs
    namespace="real"
)

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [33]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [34]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [35]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [56]:
query = "What are Allergies?"

# create the query vector
xq = embeddings.embed_query(query)

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True, namespace="real", include_values=True)
xc

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.query(vector=xq, top_k=2, include_metadata=True, namespace="real", include_values=True),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  value is not a valid dict (type=type_error.dict)